In [3]:
import sqlite3
cxn = sqlite3.connect("/home/mok/mining/jugements/cappel-mcgill/cappel.sqlite")
cs = cxn.cursor()

In [6]:
tags = []
for i, in cs.execute("select Indexation from jugements where lang='fr'").fetchall():
    tags+= [j. strip() for j in i.upper().split(";")]

In [7]:
from collections import Counter
tags = Counter(tags)

In [26]:
sum(1 for i in tags if tags[i]>=10)

763

In [28]:
from scipy import sparse as sp

In [43]:
tagsls = [ i for i in tags if tags[i]>=10 ]

In [68]:
n_docs =cs.execute("select count(*) from jugements where lang='fr'").fetchone()[0]
mat = sp.lil_matrix((len(tagsls),n_docs), dtype=bool)

In [36]:
from operator import itemgetter
import numpy as np

In [73]:
data = []
row = []
col = []
for doc, ts in cs.execute("select rowid, Indexation from jugements where lang='fr'").fetchall():
    for t in ts.split(";"):
        tt = t.upper().strip()
        if tt not in tagsls: continue
        data += [True]
        row += [tagsls.index(tt)]
        col +=[domifs.index(doc)]

mat = sp.csr_matrix((data,(row,col)), dtype=bool)

In [81]:
def is_parent(c, p):
    return np.all(np.logical_or(c, p) == p)

In [107]:
def find_parents(row):
    c=mat[row]
    s = c.sum()
    cols = c.nonzero()[1]
    return (mat.T[cols].T.sum(axis=1)==s).nonzero()[0]

In [111]:
find_parents(3)

array([  3, 745])